In [45]:
import os
from typing import List
from typing import Dict
from glob import glob
import numpy as np
import res

In [46]:
RU_INDEX_FILE = "./../a-crawling/crawling/crawling/spiders/pages/index.txt"
RU_LEMMAS_FILE = "./../b-tokens/ru_lemmas.txt"
RU_LEMMAS_TF_IDF_FOLDER = "./../d-tf-idf/ru-lemmas/"


In [47]:
RU_WEBS_DIR = "./../a-crawling/crawling/crawling/spiders/pages/ru"
ru_webs_pl = glob(f"{RU_WEBS_DIR}/**.html")

print(f"Russian websites: {len(ru_webs_pl)}")

Russian websites: 195


In [48]:
# config

# RU or EN
lang = "RU" 
webs_pl = ru_webs_pl
lemmas_file_name = RU_LEMMAS_FILE
index_file_name = RU_INDEX_FILE
tf_idf_folder = RU_LEMMAS_TF_IDF_FOLDER


In [49]:
# key - web page id (believed to be int), pages[idx] - url for that web page
pages = dict()

# key - file (web page) id, file_lemmas[idx] - lemmas for file with given id
page_lemmas = dict()

# mapping of idxes 
# key - new value of page index (matching to the row in matrix), value - initial idx of file from {index_file_name}
# why needed? 1) real idxes start from 1, 2) real idxes has 'windows' in values range
idxs_mapping = dict()

In [50]:

with open(index_file_name, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for line in lines:
        index_enrty = line.split(',')

        page_url = index_enrty[0]
        page_lang = index_enrty[1].upper()
        html_file_name = index_enrty[2]

        file_idx_match = re.search(r'(\d+)-.*\.html$', html_file_name)
        file_idx = int(file_idx_match.group(1))

        # language filter
        if (page_lang == lang):
            pages[file_idx] = page_url

print(len(pages))

177


In [51]:
page_idxes = pages.keys()
count = 0

for page_idx in page_idxes:
    idxs_mapping[count] = page_idx
    count = count + 1
    
    with open(f"{tf_idf_folder}{page_idx}.txt", 'r', encoding='utf-8') as file:
        lemmas = []
        lines = file.readlines()
        for line in lines:
            tf_idf_entry = line.split(' ')
            lemmas.append(tf_idf_entry[0])

        page_lemmas[page_idx] = lemmas

print(len(page_lemmas))
print(len(idxs_mapping))

177
177


In [52]:
all_lemmas = set()

for lemmas in page_lemmas.values():
    all_lemmas.update(lemmas)

# list of all existing lemmas
all_lemmas = list(all_lemmas)
print(len(all_lemmas))

10434


In [61]:
pages_count = len(idxs_mapping)
lemmas_count = len(all_lemmas)

# matrix
# row num - page idx, row value - vector for page
# column - lemma
matrix = np.zeros((pages_count, lemmas_count))

In [62]:
for i in range(0, pages_count):
    # trash - нужно не запутаться где какой индекс взять
    with open(f"{tf_idf_folder}{idxs_mapping[i]}.txt", 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            lemma, _, tf_idf = line.split(' ')
            if lemma in all_lemmas:
                # column
                lemma_index_in_matrix = all_lemmas.index(lemma)
                matrix[i][lemma_index_in_matrix] = float(tf_idf)
            else:
                print(f"Opps '{lemma}' not found in all_lemmas. matrix corrupted")

In [66]:
# test - значение tf_idf в файле для леммы
lemma_column = all_lemmas.index("вопрос")
matrix[idxs_mapping[10]][1675]

0.00034200550459030746

In [105]:
def find_key(dictionary, q_value):
    for key, value in dictionary.items():
        if value == q_value:
            return key
    return None  # If value not found in any keys

## [('', 1),
## ('https://taplink.cc/reztsovapolina', 1)]
pagge1 = "https://taplink.cc/reztsovapolina"
pagge2 = "https://www.google.com/maps/search/Mayo+Clinic+MN/"

page_idx = find_key(pages, pagge1)
print(page_idx)
row_page_idx = find_key(idxs_mapping, page_idx)
print(row_page_idx)

page_vector = matrix[row_page_idx]
sum(page_vector)

156
154


0.0

In [74]:
import spacy
import spacy_transformers

nlp_en = spacy.load("en_core_web_sm")
nlp_ru = spacy.load("ru_core_news_sm")

def get_tokens(text):
    if (lang == "RU"):
        doc = nlp_ru(text)
    else:
        doc = nlp_en(text)
    
    return [w for w in doc if (w.is_alpha and not w.is_stop)]

def query_to_vector(query: str) -> np.ndarray:
    vector = np.zeros(lemmas_count)
    tokens = get_tokens(query)
    #print(tokens)
    for token in tokens:
        lemma = token.lemma_
        if lemma in all_lemmas:
            lemma_index_in_matrix = all_lemmas.index(lemma)
            vector[lemma_index_in_matrix] = 1
            #print(lemma_index_in_matrix)

    return vector

In [75]:
vector = query_to_vector("вопрос подростков")
vector[1675]

1.0

In [79]:
#!pip install scipy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 527.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 3.9 MB/s eta 0:00:0000:0100:01


In [107]:
from scipy.spatial import distance as di

# returns list of turples, sorted by distance
# (url, distance)
def search(query: str) -> List:
    query_vector = query_to_vector(query)
    relevant_pages = dict()
    
    for page_row_idx, page_vector in enumerate(matrix):
        # cosine similarity between 2 vectors
        # https://medium.com/@milana.shxanukova15/cosine-distance-and-cosine-similarity-a5da0e4d9ded
        if (sum(page_vector) > 0):
            distance = 1 - di.cosine(query_vector, page_vector)
            if distance > 0:
                relevant_pages[page_row_idx] = distance

    # sort by distance
    relevant_pages_sorted = sorted(relevant_pages.items(), key=lambda item: item[1], reverse=True)

    # file idx - initial url
    search_result = [(pages[idxs_mapping[idx_to_distance[0]]], idx_to_distance[1]) 
                     for idx_to_distance in relevant_pages_sorted]

    return search_result

In [108]:
res = search("запор")

res

[('https://cuprum.media/stesnyayus-sprosit/soft-case', 0.11548489418306751),
 ('https://cuprum.media/spravochnik/obsledovanie-shitovidnoj-zhelezy',
  0.07995052320816654),
 ('https://cuprum.media/spravochnik/f-mrt', 0.04356624800386455),
 ('https://cuprum.media/spravochnik/f-mrt', 0.04356624800386455),
 ('https://cuprum.media/spravochnik', 0.03457543973694999),
 ('https://cuprum.media/lifestyle/you-are-beautiful?si=630c8755a605d5d5910f46e4',
  0.02604130777551772),
 ('https://cuprum.media/columns/nizkouglevodnye-diety', 0.02125812944041472)]

In [ ]:
class VectorSearch:
    def __init__(self):

        
    def search(self, query: str) -> List:
        query_vector = 